In [ ]:
import datetime as dt
import geopandas as gpd
import os
import fiona
import rasterio.mask
from rasterio.fill import fillnodata
from rasterstats import zonal_stats
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import rasterio
import ogr
import warnings
import json
import pandas as pd
from earthpy import clip
from shapely.geometry import JOIN_STYLE
from geopandas import GeoSeries, GeoDataFrame
from math import ceil
from shapely.ops import unary_union

import scipy.spatial
from pathlib import Path
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

# Selecting Datasets
Select the workspace, this is the folder that will be used for the outputs. 
NOTE Select an empty folder as all the files will be deleted from the workspace 

You will also have to select the three datasets used in the analysis. These are:
    
1) Administrative boundaries.

2) Agro Maps

In [ ]:
messagebox.showinfo('AGRODEM Creating base grid', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('AGRODEM Creating base grid', 'Select the administrative boundaries')
filename_admin = (filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
admin=gpd.read_file(filename_admin)

messagebox.showinfo('AGRODEM Creating base grid', 'Select the agro map')
filename_agro = (filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
agro=gpd.read_file(filename_agro)

# Select the target coordinate system


Find the appropriate target crs from:  https://epsg.io/

In [ ]:
crs = 'EPSG:3037'

# Ensuring vectors are in the same coordinate system

In [ ]:
# reprojecing to target coordinate system written above
def target_crs(vectors,crs,workspace):   
    vectors = vectors.to_crs(crs) 
    vectors.to_file(workspace, driver='ESRI Shapefile')  
    return vectors

agro_pr = agro.to_crs(crs)
admin_pr = admin.to_crs(crs)


# Fix geometries

In [ ]:
def fixgeometries(polygon):
    #creates a valid representation of a given invalid geometry without losing any of the input vertices. 
    fix = polygon.buffer(0.001)
    return fix

# Preparing agro maps

In [ ]:
#fix geometries
fixed_agro = fixgeometries(agro_pr)

#clip agro map to place of interest
clipped_agro = gpd.clip(fixed_agro,admin_pr)

#determining yield                            
clipped_agro['yield'] =  clipped_agro['product_ha']  / clipped_agro['harv_area_']   

#Calculating the area of each unit [ha]
clipped_agro['area'] =  (clipped_agro['geometry'].area)/10000

#Calculating the perimeter of each unit [km]
clipped_agro["perimeter"] = (clipped_agro["geometry"].length)/10000

# Creating base point layer

In [ ]:
def make_grid(gdf, height, cut=True):
    """
    Return a grid, based on the shape of *gdf* and on a *height* value (in
    units of *gdf*). If cut=False, the grid will not be intersected with *gdf*
    (i.e it makes a grid on the bounding-box of *gdf*).
    Parameters
    ----------
    gdf: GeoDataFrame
        The collection of polygons to be covered by the grid.
    height: Integer
        The dimension (will be used as height and width) of the ceils to create,
        in units of *gdf*.
    cut: Boolean, default True
        Cut the grid to fit the shape of *gdf* (ceil partially covering it will
        be truncated). If False, the returned grid will fit the bounding box
        of *gdf*.
    Returns
    -------
    grid: GeoDataFrame
        A collection of polygons.
    """
   
    xmin, ymin = [i.min() for i in gdf.bounds.T.values[:2]]
    xmax, ymax = [i.max() for i in gdf.bounds.T.values[2:]]
    rows = int(ceil((ymax-ymin) / height))
    cols = int(ceil((xmax-xmin) / height))

    x_left_origin = xmin
    x_right_origin = xmin + height
    y_top_origin = ymax
    y_bottom_origin = ymax - height

    res_geoms = []
    for countcols in range(cols):
        y_top = y_top_origin
        y_bottom = y_bottom_origin
        for countrows in range(rows):
            res_geoms.append((
                (x_left_origin, y_top), (x_right_origin, y_top),
                (x_right_origin, y_bottom), (x_left_origin, y_bottom)
                ))
            y_top = y_top - height
            y_bottom = y_bottom - height
        x_left_origin = x_left_origin + height
        x_right_origin = x_right_origin + height
    if cut:
        if all(gdf.eval(
            "geometry.type =='Polygon' or geometry.type =='MultiPolygon'")):
            res = GeoDataFrame(
                geometry=pd.Series(res_geoms).apply(lambda x: Polygon(x)),
                crs=gdf.crs
                ).intersection(unary_union(gdf.geometry))
        else:
            res = GeoDataFrame(
                geometry=pd.Series(res_geoms).apply(lambda x: Polygon(x)),
                crs=gdf.crs
                ).intersection(unary_union(gdf.geometry).convex_hull)
        res = res[res.geometry.type == 'Polygon']
        res.index = [i for i in range(len(res))]
        return GeoDataFrame(geometry=res)

    else:
        return GeoDataFrame(
            index=[i for i in range(len(res_geoms))],
            geometry=pd.Series(res_geoms).apply(lambda x: Polygon(x)),
            crs=gdf.crs
            )

In [ ]:
 make_grid(clipped_agro,250, cut=False)